# EDA

This notebook cleans the Chocolate Sales dataset and creates 1–2 static visuals to support our dashboard user story (sales trends over time by country).

## Why we do these checks

Before making plots, we do quick data checks that keep the dashboard stable: we confirm data types, check missing values and duplicates, and make sure key fields like Date and Amount are usable. We also create simple Year/Month fields because most dashboard views (monthly trends and filtering) depend on them.

In [102]:
import pandas as pd

DATA_PATH = "../data/raw/chocolate-sales.csv"
df_raw = pd.read_csv(DATA_PATH)
df_raw.shape

(3282, 6)

In [103]:
df_raw.head()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04/01/2022,"$5,320.00",180
1,Van Tuxwell,India,85% Dark Bars,01/08/2022,"$7,896.00",94
2,Gigi Bohling,India,Peanut Butter Cubes,07/07/2022,"$4,501.00",91
3,Jan Morforth,Australia,Peanut Butter Cubes,27/04/2022,"$12,726.00",342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24/02/2022,"$13,685.00",184


In [104]:
df_raw.info()
df_raw.isna().sum()
df_raw.duplicated().sum()

<class 'pandas.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Sales Person   3282 non-null   str  
 1   Country        3282 non-null   str  
 2   Product        3282 non-null   str  
 3   Date           3282 non-null   str  
 4   Amount         3282 non-null   str  
 5   Boxes Shipped  3282 non-null   int64
dtypes: int64(1), str(5)
memory usage: 324.8 KB


np.int64(0)

In [105]:
df = df_raw.copy()

In [106]:
# Parse Date so we can do monthly trends
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
df["Date"].isna().sum()

np.int64(0)

In [107]:
# Convert Amount to numeric for sums/plots
df["Amount"] = (
    df["Amount"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
    .astype(float)
)
df[["Amount"]].head()

,Amount
0,5320.0
1,7896.0
2,4501.0
3,12726.0
4,13685.0


In [108]:
# Add Year/YearMonth for filters and time-series grouping
df["Year"] = df["Date"].dt.year
df["YearMonth_period"] = df["Date"].dt.to_period("M")        # true time type
df["YearMonth"] = df["YearMonth_period"].astype(str)         # label like 2022-01
df["MonthName"] = df["Date"].dt.strftime("%b")               # "Jan", "Feb", ...

df[["Date", "Year", "YearMonth", "MonthName"]].head()

,Date,Year,YearMonth,MonthName
0,2022-01-04,2022,2022-01,Jan
1,2022-08-01,2022,2022-08,Aug
2,2022-07-07,2022,2022-07,Jul
3,2022-04-27,2022,2022-04,Apr
4,2022-02-24,2022,2022-02,Feb


In [109]:
from pathlib import Path

out_dir = Path("..") / "data" / "processed"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "chocolate_sales_clean.csv"
df.to_csv(out_path, index=False)

out_path

PosixPath('../data/processed/chocolate_sales_clean.csv')